<a href="https://colab.research.google.com/github/Fei-tidote/TNIC_EPSTEIN_CPAXTRA_TNICxKMUTNB/blob/main/TNIC_EPSTEIN_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMRegressor

def build_dataset():
    print("Loading datasets...")
    sales = pd.read_csv('/content/sales_history.csv')
    price_cost = pd.read_csv('/content/price_cost.csv')
    inv = pd.read_csv('/content/inventory.csv')
    comp = pd.read_csv('/content/competitor_prices.csv')
    xel = pd.read_csv('/content/XEL.csv')
    cal = pd.read_csv('/content/calendar_weather.csv')
    sku = pd.read_csv('/content/sku_master.csv')
    store = pd.read_csv('/content/store_master.csv')

    sales['date'] = pd.to_datetime(sales['date'])
    cal['date'] = pd.to_datetime(cal['date'])
    comp['date'] = pd.to_datetime(comp['date'])

    top_subs = xel.loc[xel.groupby('sku_i')['xel_ij'].idxmin()].rename(columns={'sku_i': 'sku_id', 'sku_j': 'sub_sku_id'})
    sub_prices = comp.rename(columns={'sku_id': 'sub_sku_id', 'comp_price': 'sub_comp_price'})

    master = sales.merge(price_cost, on='sku_id', how='left')
    master = master.merge(cal, on='date', how='left')
    master = master.merge(sku, on='sku_id', how='left')
    master = master.merge(store, on='store_id', how='left')

    master = master.merge(comp, on=['sku_id', 'date'], how='left')
    master = master.merge(top_subs[['sku_id', 'sub_sku_id']], on='sku_id', how='left')
    master = master.merge(sub_prices, on=['sub_sku_id', 'date'], how='left')

    master['comp_price'] = master['comp_price'].fillna(master['regular_price'])
    master['sub_comp_price'] = master['sub_comp_price'].fillna(master['regular_price'])

    return master, price_cost, inv, cal, comp, top_subs, sku, store

def generate_price():
    master, price_cost, inv, cal, comp, top_subs, sku, store = build_dataset()

    numeric_features = [
        'dow', 'is_payday', 'is_holiday', 'temp', 'rain_index',
        'regular_price', 'unit_cost', 'comp_price', 'sub_comp_price',
        'price_paid', 'income_index'
    ]

    categorical_features = [
        'store_id', 'sku_id', 'category', 'subcategory', 'brand', 'region', 'store_type'
    ]

    for col in categorical_features:
        master[col] = master[col].astype('category')

    all_features = numeric_features + categorical_features
    train_df = master.dropna(subset=all_features + ['qty']).copy()

    print("Training Identity-Aware LightGBM Model...")
    model = LGBMRegressor(n_estimators=250, learning_rate=0.05, max_depth=12, random_state=42)
    model.fit(train_df[all_features], train_df['qty'], categorical_feature=categorical_features)

    print("Executing...")
    sub = pd.read_csv('sample_submission.csv')
    sub['date_parsed'] = pd.to_datetime(sub['date'], format='%d/%m/%Y')
    cal_future = cal[cal['date'].isin(sub['date_parsed'].unique())].copy()

    results_list = []

    for (store_id, sku_id), group in sub.groupby(['store_id', 'sku_id']):
        pc = price_cost[price_cost['sku_id'] == sku_id].iloc[0]
        inv_record = inv[(inv['store_id'] == store_id) & (inv['sku_id'] == sku_id)]

        unit_cost = pc['unit_cost']
        vat_rate = pc['vat_rate']
        regular_price = pc['regular_price']
        current_inventory = inv_record['on_hand'].iloc[0] if not inv_record.empty else 9999

        group_dates = group['date_parsed']
        future_df = cal_future[cal_future['date'].isin(group_dates)].copy()

        future_df['store_id'] = store_id
        future_df['sku_id'] = sku_id
        future_df = future_df.merge(sku, on='sku_id', how='left')
        future_df = future_df.merge(store, on='store_id', how='left')

        sub_id = top_subs[top_subs['sku_id'] == sku_id]['sub_sku_id'].values
        sub_sku = sub_id[0] if len(sub_id) > 0 else sku_id

        comp_data = comp[(comp['sku_id'] == sku_id) & (comp['date'].isin(group_dates))]
        sub_comp_data = comp[(comp['sku_id'] == sub_sku) & (comp['date'].isin(group_dates))]

        future_df = future_df.merge(comp_data[['date', 'comp_price']], on='date', how='left')
        future_df = future_df.merge(sub_comp_data[['date', 'comp_price']].rename(columns={'comp_price': 'sub_comp_price'}), on='date', how='left')
        future_df['comp_price'] = future_df['comp_price'].fillna(regular_price)
        future_df['sub_comp_price'] = future_df['sub_comp_price'].fillna(regular_price)

        for col in categorical_features:
            future_df[col] = future_df[col].astype('category')

        p_min = max(unit_cost * (1 + vat_rate), regular_price * 0.70)
        p_max = regular_price * 1.30

        candidate_prices = [
            round(base + ending, 2)
            for base in range(int(np.floor(p_min)), int(np.ceil(p_max)) + 1)
            for ending in [0.00, 0.50, 0.90]
            if p_min <= round(base + ending, 2) <= p_max
        ]

        if not candidate_prices:
            best_price = regular_price
        else:
            future_df['key'] = 1
            prices_df = pd.DataFrame({'price_paid': candidate_prices, 'key': 1})
            scoring_grid = pd.merge(future_df, prices_df, on='key').drop('key', axis=1)

            scoring_grid['regular_price'] = regular_price
            scoring_grid['unit_cost'] = unit_cost

            scoring_grid['pred_qty'] = model.predict(scoring_grid[all_features])

            results = scoring_grid.groupby('price_paid').agg(total_14d_demand=('pred_qty', 'sum')).reset_index()

            results['actual_sales'] = np.where(results['total_14d_demand'] > current_inventory, current_inventory, results['total_14d_demand'])

            results['gross_profit'] = ((results['price_paid'] / (1 + vat_rate)) - unit_cost) * results['actual_sales']

            results['stockout_penalty'] = np.where(results['total_14d_demand'] > current_inventory, results['total_14d_demand'] - current_inventory, 0)

            results['simulated_score'] = results['gross_profit'] - results['stockout_penalty']

            best_price = results.loc[results['simulated_score'].idxmax(), 'price_paid']

        group_res = group.copy()
        group_res['proposed_price'] = best_price
        results_list.append(group_res)

    final_sub = pd.concat(results_list).drop(columns=['date_parsed']).sort_values('ID')
    final_sub.to_csv('submission_final.csv', index=False)
    print("SUCCESS: submission_final.csv generated!")

if __name__ == "__main__":
    generate_price()

Loading datasets...
Training Identity-Aware LightGBM Model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014037 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1244
[LightGBM] [Info] Number of data points in the train set: 216000, number of used features: 18
[LightGBM] [Info] Start training from score 9.926366
Executing...
SUCCESS: submission_final.csv generated!
